# Fuzzy Matching

In [1]:
# ! py -m pip install thefuzz

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.6/1.6 MB 6.1 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import thefuzz
from thefuzz import fuzz
from thefuzz import process

C:\Users\mounsl\AppData\Local\Temp\1\ipykernel_16048\1863003785.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [22]:
'''
Input File is expected to have columns
System
Table Name
Column Name
'''
inputFilename = 'CombinedDDmasters.xlsx'

In [25]:
fields = pd.read_excel(inputFilename, usecols=['System', 'Table Name', 'Column Name'])

In [26]:
fields

,System,Table Name,Column Name
0,Empower,Contractor,Abbr
1,P6,UNITOFMEASURE,ABBREVIATION
2,Empower,CfsrThreshold,AcAndOr
3,Empower,UserContract,AccessFlag
4,P6,ACTIVITY,ACCOUNTINGVARIANCE
...,...,...,...
4239,Empower,Element,XyEvm
4240,Empower,Contract,YellowPct
4241,Empower,ContrUnit,ZeroVal
4242,Cobra,PROGRAM,ZIP


In [27]:
fields["match results"] = fields['Column Name'].apply(lambda x: process.extract(x, fields['Column Name'], scorer=fuzz.ratio))

In [32]:
def extract_matches(df):
    new_rows = []
    for index, row in df.iterrows():
        for result in row['match results']:
            new_row = {'System':row['System'],
                    'Table Name':row['Table Name'],
                    'Column Name':row['Column Name'],
                    'Match Name':result[0],
                    'Match Ratio':result[1],
                    'Match System':fields.iloc[result[2]]['System'],
                    'Match Table':fields.iloc[result[2]]['Table Name']}
            new_rows.append(new_row)
    new_df = pd.DataFrame(new_rows, columns=['System','Table Name','Column Name','Match Name','Match Ratio','Match System','Match Table'])
    new_df = new_df[(new_df['System'] != new_df['Match System'])]
    return new_df
        
def search_term(df, term, regex=False):
    results_df = df[(df['Match Name'].str.contains(term,case=False,regex=regex))|(df['Column Name'].str.contains(term,case=False, regex=regex))]
    return results_df

In [30]:
fields = extract_matches(fields)

In [33]:
id_num = search_term(fields, 'num|id', regex=True)

In [34]:
id_num

,System,Table Name,Column Name,Match Name,Match Ratio,Match System,Match Table
896,Empower,Userr,AdminUser,RDBMSUSERID,60,P6,GLOBALSECURITY
897,Empower,Userr,AdminUser,UDFNUMBER,56,P6,UDFVALUE
926,Empower,Userr,AiRoleID,ROLEID,86,P6,RESOURCEASSIGNMENT
934,Cobra,CALCDETL,ALIAS,CalID,60,Empower,Calendar
1071,Cobra,CAWP,APPLINK,ApprID,62,Empower,ai
...,...,...,...,...,...,...,...
21099,Empower,EuwtLink,WeightID,WEIGHT,86,P6,PROJECTCODETYPE
21102,Empower,Weight,WeightID,WEIGHT,86,P6,ACTIVITYSTEP
21103,Empower,Weight,WeightID,WEIGHT,86,P6,PROJECTCODE
21104,Empower,Weight,WeightID,WEIGHT,86,P6,PROJECTCODETYPE
